In [37]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn        
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import Image, display

In [45]:
# defines linear classifier model
class LinearModel(nn.Module):
    def __init__(self):
        # define layers of net
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(400, 10)
        
        
    def forward(self, x):
        # define the forward prop function
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

In [77]:
# returns training data in tuple where 
# tuple[0] = X =  [1,20,20] tensor (20x20 input image)
# tuple[1] = Y = [] tensor (Scalar output value)
def get_train_data():
    transform = transforms.Compose([
        transforms.Resize(20), 
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    return datasets.MNIST('./data', 
                          train=True, 
                          transform=transform, 
                          target_transform=None, 
                          download=True)

In [40]:
# returns data batched according to specified batch size
def batch_data(data, batch_size):
    return torch.utils.data.DataLoader(
        data, 
        batch_size=batch_size, 
        shuffle=True)
    

In [78]:
# returns testing data in tuple where 
# tuple[0] = X =  [1,20,20] tensor (20x20 input image)
# tuple[1] = Y = [] tensor (Scalar output value)
def get_test_data():
    transform=transforms.Compose([
        transforms.Resize(20),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    return datasets.MNIST('./data',
                          train=False,
                          transform=transform,
                          target_transform=None,
                          download=True)

In [90]:
def train(model, device, train_batches, optimizer, epoch):
    model.train() # put model in training mode
    
    for batch_idx, (data, target) in enumerate(train_batches):
        batch_size = len(data) # Question: Is this weird????
        data = data.view(batch_size, 400) # reshape data to be 1 * 400
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # negative log likelihood loss
        loss.backward()
        optimizer.step()
                
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, len(train_batches.dataset),
                100. * batch_idx / len(train_batches), loss.item()))

In [69]:
def test(model, device, test_batches):
    print "--- Testing ---"
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_batches):
            batch_size = len(data)
            data = data.view(batch_size, 400)
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_batches.dataset)

    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_batches.dataset),
        100. * correct / len(test_batches.dataset)))

In [88]:
def main():
    
    device = torch.device("cpu")
    train_bs = 100 # training batch size
    test_bs = 100 # test batch size
    lr = 0.01 # learning rate
    momentum = 0.5 # momentum ??
    epochs = 5 # epochs
    
    
    train_data = get_train_data()
    train_batches = batch_data(train_data, train_bs)
    
    test_data = get_test_data()
    test_batches = batch_data(test_data, test_bs)
    
    
    model = LinearModel().to(device) # load model to cpu
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        train(model, device, train_batches, optimizer, epoch)
        test(model, device, test_batches)
    print('Done!')

    #torch.save(model, './linearclassifier.pth')
    

In [91]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.370622
Train Epoch: 1 [1000/60000 (2%)]	Loss: 1.770278
Train Epoch: 1 [2000/60000 (3%)]	Loss: 1.261707
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.966560
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.885815
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.827407
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.639584
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.661953
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.617201
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.579998
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.622330
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.615051
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.574142
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.643411
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.449528
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.659826
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.535575
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.548456
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.530815
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.374317
Train Epoch: 1 [200

Train Epoch: 3 [42000/60000 (70%)]	Loss: 0.288893
Train Epoch: 3 [43000/60000 (72%)]	Loss: 0.546860
Train Epoch: 3 [44000/60000 (73%)]	Loss: 0.260390
Train Epoch: 3 [45000/60000 (75%)]	Loss: 0.312581
Train Epoch: 3 [46000/60000 (77%)]	Loss: 0.323486
Train Epoch: 3 [47000/60000 (78%)]	Loss: 0.328567
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.262885
Train Epoch: 3 [49000/60000 (82%)]	Loss: 0.291125
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.462259
Train Epoch: 3 [51000/60000 (85%)]	Loss: 0.402523
Train Epoch: 3 [52000/60000 (87%)]	Loss: 0.398992
Train Epoch: 3 [53000/60000 (88%)]	Loss: 0.381578
Train Epoch: 3 [54000/60000 (90%)]	Loss: 0.271760
Train Epoch: 3 [55000/60000 (92%)]	Loss: 0.499289
Train Epoch: 3 [56000/60000 (93%)]	Loss: 0.324667
Train Epoch: 3 [57000/60000 (95%)]	Loss: 0.426814
Train Epoch: 3 [58000/60000 (97%)]	Loss: 0.363079
Train Epoch: 3 [59000/60000 (98%)]	Loss: 0.168284
--- Testing ---

Test set: Average loss: 0.3099, Accuracy: 9116/10000 (91%)

Train Epoch: 4 [0/60000